In [12]:
# Using Transfer Learning Models: VGG16

In [13]:
# required imports

from tensorflow.keras.layers import  Dense, Flatten

from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16

In [14]:
# loading VGG16 model
vgg = VGG16(input_shape=[224, 224] + [3], weights='imagenet', include_top=False)

# acquiring trained parameters
for layer in vgg.layers:
    layer.trainable = False
  

In [15]:
# adding a flatten layer
modified_vgg = Flatten()(vgg.output)

# adding final output layer
vgg_pred = Dense(211, activation='softmax')(modified_vgg)

# create a model object
model_vgg = Model(inputs=vgg.input, outputs=vgg_pred)

model_vgg.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [16]:
# compile the model

model_vgg.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [17]:
# required imports

from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [18]:
# using custom variations on training images as given in docs

data_gen_vgg = ImageDataGenerator(preprocessing_function=preprocess_input)

In [19]:
# extract training images from directory

training_set = data_gen_vgg.flow_from_directory(
                directory = "Currency_Images/coins/data/train",
                target_size = (224, 224),
                class_mode = 'categorical',
                batch_size = 32,
                subset = 'training'
                )

Found 6413 images belonging to 211 classes.


In [20]:
# validation set images extracted from directory

validation_set = data_gen_vgg.flow_from_directory(
        directory = 'Currency_Images/coins/data/validation',
        target_size=(224, 224),
        class_mode='categorical',
        batch_size=32
)

Found 844 images belonging to 211 classes.


In [21]:
# test images extracted from directory

test_set = data_gen_vgg.flow_from_directory(
            directory = 'Currency_Images/coins/data/test',
            target_size=(224, 224),
            class_mode='categorical',
            batch_size=32
        )

Found 844 images belonging to 211 classes.


In [22]:
# model fit on the training set

history_vgg = model_vgg.fit_generator(generator = training_set,
                              validation_data = validation_set,
                              epochs = 3
                             )

Epoch 1/3
201/201 [==============================] - 3567s 18s/step - loss: 33.1895 - accuracy: 0.5358 - val_loss: 26.6094 - val_accuracy: 0.6742
Epoch 2/3
201/201 [==============================] - 2561s 13s/step - loss: 8.7623 - accuracy: 0.8576 - val_loss: 31.2987 - val_accuracy: 0.7121
Epoch 3/3
201/201 [==============================] - 2802s 14s/step - loss: 7.2258 - accuracy: 0.9004 - val_loss: 33.2415 - val_accuracy: 0.7133


In [23]:
# final evaluation on the test set

score_vgg16 = model_vgg.evaluate_generator(test_set, verbose=2)


Instructions for updating:
Please use Model.evaluate, which supports generators.
27/27 - 209s - loss: 28.7852 - accuracy: 0.7121


In [24]:
# test loss and test accuracy

print('VGG16 Model : Test Loss is : ', score_vgg16[0])
print('VGG16 Model : Test Accuracy is : ', score_vgg16[1])

VGG16 Model : Test Loss is :  28.785228729248047
VGG16 Model : Test Accuracy is :  0.7120853066444397


In [25]:
model_vgg.save('Vgg16_model_for_currency_classification.h5')